# Aula 5 - Manipulação de df: groupby, merge e apply


### Objetivos

Apresentar como unir dataframes e realizar cálculos com dados agrupados.

____________________________

### Habilidades a serem desenvolvidas nessa aula

Ao final da aula o aluno deve:

- Saber como concatenar dataframes,
- Conseguir agrupar os dados e aplicar vários métodos à eles


____
____
____

Nas outras aulas vimos como extrair algumas informações utilizando linhas e colunas, mas e se quisessemos uma forma mais fácil de saber a média considerando um determinado grupo, por exemplo a média de sobreviventes por Pclass do Titanic?

## Como extrair informação dos dados?

In [ ]:
import pandas as pd

In [ ]:
# lê dataframe do arquivo titanic.csv 
df = pd.read_csv("data/titanic.csv")

Como faríamos para calcular a média do preço pago para viajar no titanic para cada uma das classes utilizando apenas o que aprendemos até agora?

In [ ]:
print(df[df['Pclass']==1].Fare.mean())
print(df[df['Pclass']==2].Fare.mean())
print(df[df['Pclass']==3].Fare.mean())

Ou de forma mais automática:

In [ ]:
for i in df.Pclass.sort_values().unique():
    print(f'Ticket médio da classe {i}: ', df[df['Pclass']==i].Fare.mean())

E se quisessemos calcular a média por Pclass e Sex?

### [Groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)
Assim como no SQL, no pandas também temos um método com o qual podemos agregar os dados. O `groupby` primeiro separa nossos dados em grupos definidos dentro do método,  aplica uma operação usando agregação, transformação, filtragem ou até uma função própria e, por fim, junta os resultados encontrados.
<br>

<img src="groupby.png"  style="width: 700px" >

Exemplo de agregação da coluna de sexo e aplicação a função de agregação `mean` para obter a média das alturas.

<br>

Utilizar o `groupby` é o mesmo que fazer a sequência:

   1. Dividir os dados em grupos utilizando um critério
    
   2. Aplicar uma função em cada um dos grupos separadamente
    
   3. Combinar o resultado em uma estrutura de dados

<br>
Estrutura:

`df.groupby([lista_colunas_queremos_agregar]).funcao_agregacao()`

Exemplo:

`df.groupby(['Gender]).mean()`

#### Funções de agregação
Com essas funções podemos aplicar operações estatísticas nos nossos dados. Exemplos:

`mean`, `std`, `max`, `min`, `count`, `sum`, `var`, `size`, `describe`, `first`, `last`, `nth`.

Quando queremos aplicar apenas uma dessas operações podemos chamá-las diretamente após o `groupby`:


In [ ]:
# Agrupa por Pclass e Sex e calcula a média de cada grupo


Aqui agregamos os dados por Pclass e Sex e em todas as colunas numéricas foi calculada a média. Se quiséssemos a média de apenas uma coluna poderíamos adicioná-la ao final da nossa sentença:

In [ ]:
# Queremos apenas a média de idade considerando a classe e o sexo


Ou de modo mais eficiente:

Note que `df.groupby('A').colname.mean()` é mais eficiente que `df.groupby('A').mean().colname` pois a agregação só será realizada na coluna de interesse (colname).

Quando queremos aplicar mais de uma operação chamamos o método `.agg()`

Reparem que a coluna utilizada no `groupby` virou um index do nosso df. Para convertê-la em coluna novamente temos duas formas: <br>
  1. chamar o parâmetro `as_index=False` dentro do `groupby`
  2. aplicar `.reset_index()` ao final da sentença

In [ ]:
# exemplo com as_index = False


In [ ]:
# exemplo com .reset_index()


_____________
_____________
**Exercício:** Existe diferença de sobrevivência por portão de embarque? E por diferença no preço do ticket? Porque você acha que tem essa diferença?

______________
_____________

### Apply
O método `.apply()` recebe uma função como input e aplica ela para todo o df como se fosse um loop. Se você quiser que essa função seja aplicada ao longo das colunas deve considerar axis=1 e ao longo das linhas axis=0)

Uma grande funcionalidade do pandas é que com o método `apply()` podemos aplicar uma **função** (muitas vezes, uma **função lambda**) a uma coluna ou linha de um DataFrame



Vamos selecionar a coluna de idades e aplicar uma função lambda **a todos os elementos dessa coluna** e somar um valor a ela utilizando função lambda $lambda  x: x + 2$:

Essa função lambda é equivalente a aplicar uma função do tipo:

```python

def funcao(x):

    return x + 2
```

Vamos usar uma função lambda para **extrair o sobrenome** dos nomes dos passageiros

Pra extrair o sobrenome, note que este está separada do resto do nome por vírgula.

Para perceber isso, dê uma olhada na coluna de nomes:

In [ ]:
df["Name"]

Portanto, podemos usar a função para strings `split(",")`, com quebra na vírgula, e depois selecionar o primeiro elemento da lista gerada!

Vamos aproveitar e **criar uma nova coluna da base**, com os sobrenomes!

________
________

**Exercício:** Converta os valores da coluna "Sex" de "male" para 1 e "female" para 0 utilizando o apply.

_________
_________

## Cruzamento e concatenação de bases

Também é possível fazer **cruzamento de bases** com o pandas. 

Pra quem conhece SQL: esses são os joins!

Pra quem conhece Excel: essa é uma forma de fazer o procv!

Vamos supor que temos as notas de duas provas dos alunos separas em sheets diferentes do excel e queremos juntar essa notas em um único df.

In [ ]:
# ler os dados de diferentes sheets do mesmo excel "notas.xlsx"
df1 = pd.read_excel("notas.xlsx", sheet_name="notas1")
df2 = pd.read_excel("notas.xlsx", sheet_name="notas2")

In [ ]:
df1

In [ ]:
df2

Repare que temos alunos distintos nos dois df

Diferentes tipos de join

<img src="join_exemplo2.png" text="https://towardsdatascience.com/python-pandas-dataframe-join-merge-and-concatenate-84985c29ef78"/>

O pandas possui dois métodos específicos para trabalharmos com o join de colunas entre df: `.merge()` e `.join()`. O `.merge()` fornece mais flexibilidade de trabalho e iremos utilizar e ele.

### [pd.merge()](https://pandas.pydata.org/docs/reference/api/pandas.merge.html)
pd.merge(
    left,
    right,
    how="inner",
    on=None,
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y")
)

_________________________
_________________________
**Exercício:** Tente fazer um inner, left e right join e analise as diferenças nos resultados.

_________________________
_________________________

### [pd.concat()](https://pandas.pydata.org/docs/reference/api/pandas.concat.html)
Diferente do `.merge()` e `.join()` que operam apenas com colunas, com o `.concat()` podemos especificar se queremos concatenar em linhas ou colunas.
Na concatenação de colunas o `.concat()` somente considera o index dos df e, por isso, não podemos especificar colunas como feito com o `.merge()`.

`pd.concat(
    objs,
    axis=0,
    join="outer",
    ignore_index=False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    copy=True,
)`

Repare que ao concatenar diretamente pelo index ele juntou o aluno obi wan com o anakin. 

Ao concatenar dois df nas linhas, o `.concat()` irá considerar o nome das colunas. Se temos colunas com nomes distintos e utilizamos o parâmetro join='inner', ele irá ignorar essas colunas: 

Para que ele considere todas as colunas utilizamos o argumento 
```python 
join="outer" 
```

## Exercícios

1. Considere a existência de três tabelas distintas:
* customer.csv : Possui a informação dos clientes em duas colunas: customer id  customer name
* products.csv : Conté informação dos produtos vendidos pela empresa em três colunas - p_id (product id), product (name) e price
* sales.csv : Contém informações das vendas realizadas em seis colunas - sale_id, c_id (customer id), p_id (product_id), qty (quantity sold), store (name)

Conhecendo as bases e utilizando os métodos de concatenação de bases responda:


a) Quais produtos não foram vendidos?

b) Quantos clientes não realizaram uma compra? 

c) Liste a quantidade vendida e o faturamento de cada produto 

d) Liste a quantidade vendida de cada produto por loja

e) Qual loja teve maior faturamento?

f) Qual produto foi o mais vendido?

## Referências
https://pandas.pydata.org/docs/user_guide/groupby.html <br>
https://pandas.pydata.org/docs/user_guide/merging.html <br> 
https://towardsdatascience.com/python-pandas-dataframe-join-merge-and-concatenate-84985c29ef78 <br>
[When to use pandas transform function](https://towardsdatascience.com/when-to-use-pandas-transform-function-df8861aa0dcf) <br>
[Compara a performance entre várias formas de iterar em um df. Vai desde o for até apply e transform](https://youtu.be/rsyvErL0Bo8) <br>

## Material extra

### Outros parâmetros do groupby

* as_index (que passará a ser chamado de "group_keys")
* sort
* dropna # exclui nans nas keys

Em todas o default do python é True.

In [ ]:
df.groupby('Pclass', sort=False, as_index=False)["Fare"].mean()

In [ ]:
# dropna
df_list = [[1, 2, 3], [1, None, 4], [2, 1, 3], [1, 2, 2]]
df_dropna = pd.DataFrame(df_list, columns=["a", "b", "c"])
df_dropna

In [ ]:
# Default ``dropna`` is set to True, which will exclude NaNs in keys
df_dropna.groupby(by=["b"]).sum()

In [ ]:
df_dropna.groupby(by=["b"], dropna=False).sum()

Repare que podemos chamar qualquer função do `pd.Series` ou  do `numpy`

In [ ]:
df.groupby(["Survived"]).mean()

E se quiséssemos criar uma coluna nova que contenham o valor médio do Fare por Pclass?

### Criando coluna com dado agregado

Queremos que todas as pessoas da primeira classe tenham o valor 84.15 nessa nova coluna, todas da segunda classe tenham o valor 20.66 e da terceira classe 13.67. <br>
Podemos tentar:

In [ ]:
df.groupby('Pclass')[["Fare"]].mean()

,Fare
Pclass,
1,84.154687
2,20.662183
3,13.675550


In [ ]:
df["Fare_Mean"] = df.groupby('Pclass')["Fare"].mean()

df.head(7)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,84.154687
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,20.662183
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,13.675550
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,NaN
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,NaN
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,NaN


Xiiii... deu ruim...
<br>
<br>


#### Transformação dos dados agregados
Ao aplicarmos o método [`.transform()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.transform.html) temos como retorno um objeto com o mesmo index do df de origem contendo a transformação realizada para cada uma das linhas. Dessa forma podemos utilizar esse método e apenas criar uma coluna nova no nosso df.
<br>

Ele será muito **útil na criação de novas features** para os modelos.

In [ ]:
df.groupby('Pclass')[["Fare"]].transform('mean')

,Fare
0,13.675550
1,84.154687
2,13.675550
3,84.154687
4,13.675550
...,...
886,20.662183
887,84.154687
888,13.675550
889,84.154687


In [ ]:
df["Fare_Mean"] = df.groupby('Pclass')["Fare"].transform('mean')
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,13.675550
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,84.154687
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,13.675550
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,84.154687
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,13.675550
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,13.675550
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,84.154687
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,13.675550
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,13.675550
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,20.662183


Podemos aplicar tanto as operações mencionadas na agregação quanto uma função `lambda`:

In [ ]:
df['variacao_max_min'] = df.groupby('Pclass')[["Fare"]].transform(lambda x: x.max() - x.min())
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_Mean,variacao_max_min
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,13.675550,69.5500
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,84.154687,512.3292
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,13.675550,69.5500
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,84.154687,512.3292
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,13.675550,69.5500
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,13.675550,69.5500
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,84.154687,512.3292
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,13.675550,69.5500
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,13.675550,69.5500
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,20.662183,73.5000


Ou até mesmo passar funções construídas:

In [ ]:
def funcao_max_menos_min(x):
    return x.max() - x.min()

In [ ]:
df.groupby('Pclass')[["Fare"]].transform(funcao_max_menos_min)

,Fare
0,69.5500
1,512.3292
2,69.5500
3,512.3292
4,69.5500
...,...
886,73.5000
887,512.3292
888,69.5500
889,512.3292


_________________________
_________________________
**Exercício:** Crie uma coluna com a média de Fare e outra com a média de idade para cada classe da coluna Survived. Você consegue fazer isso de uma única vez?

In [ ]:
# Resposta
df[['fare_medio', 'age_medio']]= df.groupby('Survived')[['Fare', 'Age']].transform('mean')

_________________________
_________________________

### Função Lambda
Uma função lambda nada mais é que uma **forma alternativa de declarar uma função**, de um jeito mais direto

In [ ]:
# função que retorna o dobro de um número usando def
def dobro(x):
    return 2*x

dobro(2)

In [ ]:
# função que retorna o dobro de um número usando lambda x
calcula_dobro = lambda x: 2*x
calcula_dobro(6)

### Transform X Apply

Função apply:

- É frequentemente usada para transformações mais complexas ou quando a operação envolve várias colunas ou linhas.

Função transform:

- A função transform é usada principalmente para transmitir valores agregados de volta para as linhas originais.
- É comumente usada com operações de groupby para realizar transformações enquanto mantém a forma original do DataFrame.

Resumindo, **use apply quando você precisa realizar operações mais complexas no seu DataFrame ou Series, e use transform quando quiser transmitir resultados agregados de volta para os dados originais, especialmente ao trabalhar com operações de groupby**. 

Para mais informações sobre essas diferenças consultar Material de Aprofundamento.


In [ ]:
# Criando um DataFrame de exemplo
data = {'Nome': ['Alice', 'Bob', 'Charlie', 'David'],
        'Idade': [25.0, 30, 22, 28],
        'Salário': [50000, 60000, 45000, 55000]}
dframe = pd.DataFrame(data)

# Usando apply para calcular um novo campo com bônus baseado no salário
def calcular_bonus(salario):
    if salario >= 55000:
        return 1000
    else:
        return 500

dframe['Bônus'] = dframe['Salário'].apply(calcular_bonus)
dframe['Bônus_transform'] = dframe['Salário'].transform(calcular_bonus)

# Usando transform para calcular a média de idade por bônus
dframe['Média_Idade_por_Bônus'] = dframe.groupby('Bônus')['Idade'].transform('mean')

def total_recebimentos(x):
    return x['Bônus'] + x["Salário"] 

dframe['Recebimentos'] = dframe.apply(total_recebimentos, axis=1)
# dframe['Recebimentos'] = dframe.transform(total_recebimentos, axis=1) # esse não vai funcionar

dframe

Ambos podem receber

- uma função (`np.sqrt`),
- uma string de função (`'sqrt'`)
- uma lista de função (`[np.sqrt, np.exp]`)
- uma lista de string de função (`['sqrt', 'exp']`)
- um dicionário de função (`{'A': np.sqrt, 'B': np.exp}`)


In [ ]:
dframe.Idade.transform(['sqrt', 'exp'])
dframe.Idade.apply(['sqrt', 'exp'])

In [ ]:
dframe.Idade.transform([np.sqrt, np.exp])
dframe.Idade.apply([np.sqrt, np.exp])

In [ ]:
# A dict of axis labels -> function
dframe.Idade.transform({
    'A': np.sqrt,
    'B': np.exp,
})

# A dict of axis labels -> function
dframe.Idade.apply({
    'A': np.sqrt,
    'B': np.exp,
})

`.transform()` não produz resultados agregados diretamente. A menos que vc utilize o `groupby` antes.

In [ ]:
# dframe = pd.DataFrame({'A': [1,2,3], 'B': [10,20,30] })
dframe[['Idade']].apply(lambda x: x.sum())

In [ ]:
dframe[['Idade']].transform(lambda x:x.sum())

`apply()` funciona com múltiplas séries de uma única vez enquanto no `transform()` só é permitido usar uma única Series.

In [ ]:
def subtract_two(x):
    return x['Idade'] - x['Salário']
dframe.apply(subtract_two, axis=1)

In [ ]:
dframe.transform(subtract_two, axis=1)

### Filtros em dados agregados
O [`.filter()`](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.filter.html?highlight=filter#pandas.core.groupby.DataFrameGroupBy.filter) retorna apenas um subset do nosso df e é um filtro para ser aplicado após um agrupamento de dados. Aqui podemos aplicar filtros mais elaborados do que os vistos na última aula. <br>


In [ ]:
# Filtrar grupos com média de idade acima de 25
df.groupby('Pclass').filter(lambda x: x['Age'].mean() > 25)

_________________________
_________________________
**Exercício:** Vamos supor que durante a viagem do titanic, o time de hapiness quisesse promover uma jogatina para os grupos (segmentado por classe e sexo) que na média pagaram mais de 20 dólares para entrar. Como podemos filtrar nosso df para termos apenas os passageiros que pertecem a essas segmentações escolhidas?

In [ ]:
df_filtrado = df.groupby(['Pclass','Sex']).filter(lambda x: x['Fare'].mean()>20)
df_filtrado.head()

_________________________
_________________________

## [Avaliação anônima](https://forms.gle/tShxhxNYhvi6ZmQm8)